In [1]:
import tensorflow as tf
import numpy as np
import torch
import random
import pandas as pd
import os, sys
import time
import datetime
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss

# random.seed(42)
torch.manual_seed(42) 
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM3-32GB


########################### Load dataset ###########################

In [2]:
X_train = pd.read_csv("path_to/female_only.train.en", header=None)
X_val = pd.read_csv("path_to/female_only.dev.en", header=None)
X_train = X_train[0].values
X_val = X_val[0].values

########################### tokenize dataset ###########################

In [3]:
# from transformers import BertTokenizer, AdamW, BertConfig, BertForPreTraining

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
vocabs = tokenizer.get_vocab()
seq_len = 40
def tokenize(data):
    

    # Load the BERT tokenizer.
    print('Loading BERT tokenizer...')

    # Training Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    attention_masks = []
    target_labels = []

    # For every sentence...
    for k, sent in enumerate(data):
        encoded_dict = tokenizer.encode_plus(
                            str(sent),                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = seq_len,           # Pad & truncate all sentences.
                            truncation=True,
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        
        
    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks


In [4]:

train_input_ids, train_attention_masks = tokenize(X_train)
val_input_ids, val_attention_masks = tokenize(X_val)

print('Original: ', X_train[0])
print('Token IDs:', train_input_ids[0])



from torch.utils.data import TensorDataset

train_dataset = TensorDataset(train_input_ids, train_attention_masks)
val_dataset = TensorDataset(val_input_ids, val_attention_masks)

print('{:>5,} training samples'.format(len(train_dataset)))
print('{:>5,} validation samples'.format(len(val_dataset)))


Loading BERT tokenizer...


/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Loading BERT tokenizer...
Original:  atmosphere and service was also great !
Token IDs: tensor([ 101, 7224, 1998, 2326, 2001, 2036, 2307,  999,  102,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])
100,000 training samples
1,000 validation samples


In [5]:
torch.save(train_dataset, 'generate_neutral_latent_embedding_train_dataset.pt')
torch.save(val_dataset, 'generate_neutral_latent_embedding_val_dataset.pt')


In [6]:
# train_dataset = torch.load('generate_neutral_latent_embedding_train_dataset.pt')
# val_dataset = torch.load('generate_neutral_latent_embedding_val_dataset.pt')


########################### Load model ###########################

In [7]:
from bert_model import BertForMaskedLM_bias_neutral_sentence_embedder

model = BertForMaskedLM_bias_neutral_sentence_embedder.from_pretrained("bert-base-uncased")

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM_bias_neutral_sentence_embedder: ['bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.7.attention.self.value.bias', 'bert.encoder.layer.8.attention.output.dense.bias', 'bert.encoder.layer.6.intermediate.dense.bias', 'bert.encoder.layer.3.attention.self.value.bias', 'bert.encoder.layer.7.attention.self.query.bias', 'bert.encoder.layer.7.attention.self.query.weight', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.10.attention.self.value.bias', 'bert.encoder.layer.11.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.attention.output.dense.bias', 'bert.encoder.layer.8.attention.self.query.bias', 'bert.encoder.layer.7.attention.output.dense.bias', 'bert.encoder.layer.7.output.LayerNorm.bias', 'bert.encoder.layer.5.attention.self.key.bias', 'bert.encoder.layer.5.attention.self.value.weight', 'bert.encoder.layer.4.output.LayerNorm.weight',

BertForMaskedLM_bias_neutral_sentence_embedder(
  (bias_mitigation_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

########################### optimization, training, and validation functions ###########################


In [8]:
seq_len = 40
vocab_length = 30522
from transformers.optimization import AdamW
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

# # For validation the order doesn't matter, so we'll just read them sequentially.
# test_dataloader = DataLoader(
#             test_dataset, # The validation samples.
#             sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
#             batch_size = batch_size # Evaluate with this batch size.
#         )


from transformers.optimization import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)



import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))



import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_masked_accuracy(preds, labels):
#     pred_score = preds
    logits_argmax = np.array([np.argmax(l, axis=1) for l in preds])
    pred_flat = logits_argmax.flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# # Function to calculate the accuracy of our predictions vs labels
# from torch.nn import MSELoss
# MSE_loss_fct = MSELoss()
# def flat_class_accuracy(preds):
#     labels = torch.FloatTensor(np.array([0.5]*len(preds)))
#     bias_loss = MSE_loss_fct(preds.view(-1), labels.view(-1))
#     return bias_loss


# Function to calculate the accuracy of our predictions vs labels
def flat_class_accuracy(preds):
    labels = np.array([0]*len(preds))
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
#     print(pred_flat)
#     print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)



import random
import numpy as np
from tqdm.notebook import tqdm
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
def train(model, optimizer, losslogger, start_epoch, epochs, run_id, train_dataloader, validation_dataloader, checkpoint_name):
    # Set the seed value all over the place to make this reproducible.
    seed_val = 42

    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    # We'll store a number of quantities such as training and validation loss, 
    # validation accuracy, and timings.
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()

    # For each epoch...
    t = tqdm(range(start_epoch,epochs))

    for epoch_i in t:

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0
        total_bias_loss = 0
        total_mlm_loss = 0

        # Put the model into training mode. Don't be mislead--the call to 
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)

                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the 
            # `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)

            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because 
            # accumulating the gradients is "convenient while training RNNs". 
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()        

            # Perform a forward pass (evaluate the model on this training batch).
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # It returns different numbers of parameters depending on what arguments
            # arge given and what flags are set. For our useage here, it returns
            # the loss (because we provided labels) and the "logits"--the model
            # outputs prior to activation.
#             loss, logits, class_logits = model(b_input_ids,
            mlm_loss, bias_loss, loss, bias_logits, sentence_emb, similar_sentence_pooled_output = model(b_input_ids, 
                                 token_type_ids=None, 
                                 attention_mask=b_input_mask,
                                 labels=b_input_ids,
                                              )

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            
#             print(bias_logits)
            
            total_mlm_loss += mlm_loss.item()
            total_bias_loss += bias_loss.item()
            total_train_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

            
        # Calculate the average mlm loss over all of the batches.
        avg_mlm_loss = total_mlm_loss / len(train_dataloader) 
        
        # Calculate the average bias loss over all of the batches.
        avg_bias_loss = total_bias_loss / len(train_dataloader) 
        
        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)            

        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        # set in logs
        df = pd.DataFrame()
        df['chackpoint_name'] = pd.Series(checkpoint_name)
        df['epoch'] = pd.Series(epoch_i)
        df['Loss'] = pd.Series(loss.data.item())
        df['run'] = run_id
        losslogger = losslogger.append(df)
        
        print("")
        print("  Average mlm loss: {0:.2f}".format(avg_mlm_loss))
        print("  Average bias loss: {0:.2f}".format(avg_bias_loss))
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))
        state = {'epoch': epoch_i + 1, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict(), 'losslogger': losslogger, }
        torch.save(state, f'{checkpoint_name}')

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        
        # Tracking variables 
        total_eval_masked_accuracy = 0
        total_eval_class_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0


        # Evaluate data for one epoch
        for batch in validation_dataloader:

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using 
            # the `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_target_labels = batch[0].to(device)



            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():        

                # Forward pass, calculate logit predictions.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # Get the "logits" output by the model. The "logits" are the output
                # values prior to applying an activation function like the softmax.
#                 (loss, logits, class_logits) = model(b_input_ids,
                (mlm_loss, bias_loss, loss, bias_prob, sentence_emb, similar_sentence_pooled_output) = model(b_input_ids,
                                 token_type_ids=None, 
                                 attention_mask=b_input_mask, 
                                 labels=b_input_ids,
                                                    )
    
    #             calculate MLM evaluation
            # Accumulate the validation loss.
            total_eval_loss += loss.item()

            # Move logits and labels to CPU
#             masked_logitss = masked_logits.detach().cpu().numpy()
            b_target_labelss = b_target_labels.to('cpu').numpy()

            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
#             total_eval_masked_accuracy += flat_masked_accuracy(masked_logitss, b_target_labelss)
            
#             calculte bias evaluation
            # Move logits and labels to CPU
            bias_probb = bias_prob.detach().cpu().numpy()

            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_eval_class_accuracy += flat_class_accuracy(bias_probb)


        # Report the final accuracy for this validation run.
#         avg_val_masked_accuracy = total_eval_masked_accuracy / len(validation_dataloader)
        avg_val_class_accuracy = total_eval_class_accuracy / len(validation_dataloader)
#         print("  Masked Accuracy: {0:.2f}".format(avg_val_masked_accuracy))
        print("  Class Accuracy: {0:.2f}".format(avg_val_class_accuracy))
        
         # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)


        # Measure how long the validation run took.
        validation_time = format_time(time.time() - t0)

        print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
#                 'Valid. Masked Accur.': avg_val_masked_accuracy,
                'Valid. Class Accur.': avg_val_class_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )

#             # Accumulate the validation loss.
#             total_eval_loss += loss.item()
            
#             logits = logits.detach().cpu().numpy()
#             label_ids = b_input_labels.detach().cpu().numpy()

#             # Move logits and labels to CPU
#             class_logits = class_logits.detach().cpu().numpy()
#             b_class_labels = b_class_labels.detach().cpu().numpy()
# #             class_labels2 = np.array([0.5]*len(b_class_labels))

#             # Calculate the accuracy for this batch of test sentences, and
#             # accumulate it over all batches.
#             total_eval_accuracy += flat_accuracy(class_logits, b_class_labels)
# #             MSE_loss_fct = MSELoss()
# #             m = nn.Softmax(dim=1)
# #             norm_bias_score = m(class_logits)
# #             norm_bias_score = norm_bias_score[:,:1].view(-1)
# # #             print(norm_bias_score)
# #             total_eval_accuracy += MSE_loss_fct(norm_bias_score, class_labels2)



#         # Report the final accuracy for this validation run.
#         avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
#         print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

#         # Calculate the average loss over all of the batches.
#         avg_val_loss = total_eval_loss / len(validation_dataloader)

#         # Measure how long the validation run took.
#         validation_time = format_time(time.time() - t0)

#         print("  Validation Loss: {0:.2f}".format(avg_val_loss))
#         print("  Validation took: {:}".format(validation_time))

#         # Record all statistics from this epoch.
#         training_stats.append(
#             {
#                 'epoch': epoch_i + 1,
#                 'Training Loss': avg_train_loss,
#                 'Valid. Loss': avg_val_loss,
#                 'Valid. Accur.': avg_val_accuracy,
#                 'Training Time': training_time,
#                 'Validation Time': validation_time
#             }
#         )

# #     print("")
# #     print("Training complete!")

# #     print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
          
    
    
    
def load_checkpoint(model, optimizer, losslogger, filename):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = 0
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        losslogger = checkpoint['losslogger']
        print("=> loaded checkpoint '{}' (epoch {})"
                  .format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model, optimizer, start_epoch, losslogger          
          
          
          
          


########################### Start training with checkpoint ###########################

In [9]:
# first training epoch
# Start
start_epoch = 0

# Logger
losslogger = pd.DataFrame()

# Checkpoint name
checkpoint_name = 'checkpoint.pth.tar'

train(model, optimizer, losslogger, start_epoch, epochs, 0, train_dataloader, validation_dataloader, checkpoint_name)
time.sleep(8)


======== Epoch 1 / 2 ========
Training...
  Batch    40  of  3,125.    Elapsed: 0:00:06.
  Batch    80  of  3,125.    Elapsed: 0:00:12.
  Batch   120  of  3,125.    Elapsed: 0:00:17.
  Batch   160  of  3,125.    Elapsed: 0:00:23.
  Batch   200  of  3,125.    Elapsed: 0:00:29.
  Batch   240  of  3,125.    Elapsed: 0:00:34.
  Batch   280  of  3,125.    Elapsed: 0:00:40.
  Batch   320  of  3,125.    Elapsed: 0:00:46.
  Batch   360  of  3,125.    Elapsed: 0:00:51.
  Batch   400  of  3,125.    Elapsed: 0:00:57.
  Batch   440  of  3,125.    Elapsed: 0:01:03.
  Batch   480  of  3,125.    Elapsed: 0:01:08.
  Batch   520  of  3,125.    Elapsed: 0:01:14.
  Batch   560  of  3,125.    Elapsed: 0:01:20.
  Batch   600  of  3,125.    Elapsed: 0:01:25.
  Batch   640  of  3,125.    Elapsed: 0:01:31.
  Batch   680  of  3,125.    Elapsed: 0:01:37.
  Batch   720  of  3,125.    Elapsed: 0:01:43.
  Batch   760  of  3,125.    Elapsed: 0:01:48.
  Batch   800  of  3,125.    Elapsed: 0:01:54.
  Batch   840  of

In [10]:
torch.save(model, "generate_neutral_latent_embedding_model.pth")

########################### Continue training from checkpoint ###########################

In [11]:
# # Load and continute train // run 1
# losslogger = pd.DataFrame()
# model, optimizer, start_epoch, losslogger = load_checkpoint(model, optimizer, losslogger, filename=checkpoint_name)
# !rm 'checkpoint.pth.tar'
# train(model, optimizer, losslogger, start_epoch, epochs, 1, train_dataloader, validation_dataloader, checkpoint_name)
# time.sleep(8)
